In [ ]:
from pyhdf.SD import SD,SDC
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import os
PROJECT_DIR = "/Users/zbutler/research/fire_prediction/"
os.chdir(PROJECT_DIR)
from util.tile2latlon import tile2latlon, latlon2tile
os.chdir("data/burnt/modis")
%matplotlib inline

In [ ]:
# First, let's just look at the dates of each of the files in each of the folders
dr2dates = dict()
dr2fracs = dict()
for dr in os.listdir('.'):
    dates = []
    fracs = []
    for filename in os.listdir('./%s' % dr):
        map_year = int(filename.split('.')[0][1:5])
        map_day = int(filename.split('.')[0][5:])
        frac = map_year + (float(map_day) / 365)
        dates.append((map_year, map_day))
        fracs.append(frac)
    dr2dates[dr] = dates
    dr2fracs[dr] = fracs

In [ ]:
tester = np.array(dr2fracs['h10v02'])
for i,(dr, fracs) in enumerate(dr2fracs.iteritems()):
    plt.plot(fracs, np.zeros(len(fracs)) + i, 'rs')
    if (np.array(fracs) == tester).all():
        eq = 1
    else:
        eq = 0
    print "cell %d: %s (%d records, eq = %d)" %(i, dr, len(fracs), eq)
plt.show()


In [ ]:
# Now, let's plot all the tiles next to each other in a subplot
time_str = 'A2013213'  # Collect data from 2013 
hstart = 9
hend = 11
vstart = 2
vend = 3
f, axarr = plt.subplots(vend-vstart+1, hend-hstart+1)
for h in xrange(hstart, hend+1):
    for v in xrange(vstart, vend+1):
        filename = glob("./h%.2dv%.2d/%s*" % (h, v, time_str))[0]
        hdf = SD(filename, SDC.READ)
        axarr[v-vstart, h-hstart].imshow(hdf.select("Burn Date")[:])
plt.show()

time_str_2 = 'A2013335'  # Collect data from 2013 
f, axarr = plt.subplots(vend-vstart+1, hend-hstart+1)
for h in xrange(hstart, hend+1):
    for v in xrange(vstart, vend+1):
        filename = glob("./h%.2dv%.2d/%s*" % (h, v, time_str_2))[0]
        hdf = SD(filename, SDC.READ)
        axarr[v-vstart, h-hstart].imshow(hdf.select("Burn Date")[:])
plt.show()

In [ ]:
# use tile2latlon to see the borders we're plotting rn--is it all of Alaska?
mat = hdf.select("Burn Date")[:]
n_pix = mat.shape[0]
min_lat_1, min_lon_1 = tile2latlon(9,2,0,0,n_pix)
print (min_lat_1, min_lon_1-360)
min_lat_2, min_lon_2 = tile2latlon(9,2,n_pix-1,n_pix-1,n_pix)
print (min_lat_2, min_lon_2-360)
min_lat_1, min_lon_1 = tile2latlon(9,2,0,n_pix-1,n_pix)
print (min_lat_1, min_lon_1-360)
min_lat_2, min_lon_2 = tile2latlon(9,2,n_pix-1,0,n_pix)
print (min_lat_2, min_lon_2-360)
min_lat_1, min_lon_1 = tile2latlon(9,3,0,0,n_pix)
print (min_lat_1, min_lon_1-360)
min_lat_2, min_lon_2 = tile2latlon(9,3,n_pix-1,n_pix-1,n_pix)
print (min_lat_2, min_lon_2-360)
min_lat_1, min_lon_1 = tile2latlon(9,3,0,n_pix-1,n_pix)
print (min_lat_1, min_lon_1-360)
min_lat_2, min_lon_2 = tile2latlon(9,3,n_pix-1,0,n_pix)
print (min_lat_2, min_lon_2-360)
min_lat, min_lon = tile2latlon(9, 3, 0, n_pix-1, n_pix)
max_lat, max_lon = tile2latlon(11, 2, n_pix-1, 0, n_pix)
print "Our bb1: " + str((min_lat, max_lat, min_lon-360, max_lon-360))

ak_bb = [55, 71, -165, -138]

In [ ]:
# ok this is annoying. let's try going the opposite way--figure out which tile each point of
# the ak_bb is in
tr_h, tr_v = latlon2tile(ak_bb[1], ak_bb[3])
br_h, br_v = latlon2tile(ak_bb[0], ak_bb[3])
tl_h, tl_v = latlon2tile(ak_bb[1], ak_bb[2])
bl_h, bl_v = latlon2tile(ak_bb[0], ak_bb[2])

print "top right: " + str((tr_h, tr_v))
print "bottom right: " + str((br_h, br_v))
print "top left: " + str((tl_h, tl_v))
print "bottom left: " + str((bl_h, bl_v))

In [ ]:
# use dumb haxx to see what we have rn
ak_lats = []
ak_lons = []
for h in xrange(9,12):
    for v in xrange(2,4):
        for i in xrange(n_pix):
            for j in xrange(n_pix):
                lat,lon = tile2latlon(h, v, i, j, n_pix)
                if ak_bb[0] < lat < ak_bb[1] and ak_bb[2] < lon-360 < ak_bb[3]:
                    ak_lats.append(lat)
                    ak_lons.append(lon)
        print "finished h %d v %d" %(h,v)

In [ ]:
n_pts = len(ak_lats)
ak_lats = np.array(ak_lats, dtype='float')
ak_lons = np.array(ak_lons, dtype='float') - 360
print "%d pts" %(n_pts)
print "min lat: %f, max lat: %f" %(min(ak_lats), max(ak_lats))
print "min lon: %f, max lon: %f" %(min(ak_lons), max(ak_lons))
plt.scatter(ak_lons[0:n_pts:4], ak_lats[0:n_pts:4], c='k')
plt.xlim(ak_bb[2], ak_bb[3])
plt.ylim(ak_bb[0], ak_bb[1])
plt.show()

In [ ]:
print np.min(hdf.select("First Day")[:])
print np.max(hdf.select("First Day")[:])
print np.min(hdf.select("Last Day")[:])
print np.max(hdf.select("Last Day")[:])

In [ ]:
filename_2 = glob("data/burnt/modis/h10v02/A2013182.*")[0]
hdf_2 = SD(filename_2, SDC.READ)
print np.min(hdf_2.select("First Day")[:])
print np.max(hdf_2.select("First Day")[:])
print np.min(hdf_2.select("Last Day")[:])
print np.max(hdf_2.select("Last Day")[:])
print np.unique(hdf_2.select("Burn Date")[:])

In [ ]:
#os.chdir("../../..")
from util.daymonth import day2monthday
print day2monthday(213)

In [ ]:
# How much overlap do we have between consecutive months?
fn_h10v02_jul = glob("data/burnt/modis/h10v02/A2013182.*")[0]
hdf_jul = SD(fn_h10v02_jul, SDC.READ)
fn_h10v02_aug = glob("data/burnt/modis/h10v02/A2013213.*")[0]
hdf_aug = SD(fn_h10v02_aug, SDC.READ)

print "%d burnt pixels in july" %np.sum(hdf_jul.select("Burn Date")[:] > 0)
print "%d burnt pixels in august" %np.sum(hdf_aug.select("Burn Date")[:] > 0)
combo = np.logical_and(hdf_jul.select("Burn Date")[:] > 0, hdf_aug.select("Burn Date")[:] > 0)
print "%d burned in both" %np.sum(combo)

In [ ]:
start_of_month_arr = ['001','032','060','091','121','152','182','213','244','274','305','335']
overall_arr = np.zeros((n_pix, n_pix))
for month,daystr in enumerate(start_of_month_arr):
    fn = glob("data/burnt/modis/h10v02/A2010%s.*" % daystr)[0]
    hdf_data = SD(fn, SDC.READ).select("Burn Date")[:]
    print "%d pixels burnt in month %d" %(np.sum(hdf_data > 0), month)
    if month > 0:
        last_month_overlap = np.logical_and(hdf_data > 0, last_hdf > 0)
        print "%d pixels overlap with last month in month %d" %(np.sum(last_month_overlap), month)
    overall_overlap = np.logical_and(hdf_data > 0, overall_arr > 0)
    print "%d overlap with past in month %d" %(np.sum(overall_overlap), month)
    overall_arr = np.logical_or(overall_arr, hdf_data > 0)
    last_hdf = hdf_data

In [ ]:
bin(hdf_jul.select("QA")[:][-10,-10])